In [6]:
import sys
import os

# In Jupyter, __file__ is not defined. Use the current working directory instead.
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

In [7]:
from legacy.util.fetch import fetch_url
from bs4 import BeautifulSoup
import pandas as pd

In [8]:
base_url = "https://www.krollcorp.com/"

In [9]:
content = fetch_url(base_url)

Status Code: 200


In [10]:
soup = BeautifulSoup(content, 'html.parser')

In [11]:
soup

<!DOCTYPE html>

<html lang="en">
<head>
<script>
    var LOCALE = 'en\u002DUS';
    var BASE_URL = 'https\u003A\u002F\u002Fwww.krollcorp.com\u002F';
    var require = {
        'baseUrl': 'https\u003A\u002F\u002Fwww.krollcorp.com\u002Fstatic\u002Fversion1751813609\u002Ffrontend\u002FHumanElement\u002Fkrollcorp\u002Fen_US'
    };</script> <meta charset="utf-8"/>
<meta content="Home page" name="title"/>
<meta content="Shop wholesale survival and law enforcement gear at Kroll. Stock your shelves with Kroll exclusive brands and products available now." name="description"/>
<meta content="INDEX,FOLLOW" name="robots"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<meta content="telephone=no" name="format-detection"/>
<title>Home page</title>
<link href="https://www.krollcorp.com/static/version1751813609/frontend/HumanElement/krollcorp/en_US/mage/calendar.min.css" media="all" rel="stylesheet" type="text/css"/>
<link href="https://www.krollcorp.com/static/version17518

In [12]:
navigation_bar = soup.find(class_='navigation')

navigation_bar

<nav class="navigation" data-action="navigation">
<ul data-mage-init='{"menu":{"responsive":true, "expanded":true, "position":{"my":"left top","at":"left bottom"}}}'>
<li class="level0 nav-1 category-item first level-top parent"><a class="level-top" href="https://www.krollcorp.com/clothing.html"><span>Clothing</span></a><ul class="level0 submenu"><li class="level1 nav-1-1 category-item first parent"><a href="https://www.krollcorp.com/clothing/tops.html"><span>Tops</span></a><ul class="level1 submenu"><li class="level2 nav-1-1-1 category-item first"><a href="https://www.krollcorp.com/clothing/tops/short-sleeve.html"><span>Short Sleeve</span></a></li><li class="level2 nav-1-1-2 category-item"><a href="https://www.krollcorp.com/clothing/tops/long-sleeve.html"><span>Long Sleeve</span></a></li><li class="level2 nav-1-1-3 category-item"><a href="https://www.krollcorp.com/clothing/tops/polos.html"><span>Polos</span></a></li><li class="level2 nav-1-1-4 category-item"><a href="https://www.kroll

In [17]:
category_list = []

# Find the main category (in this case: Clothing)
main_cat_li = navigation_bar.find_all('li', class_='level0')
for cat in main_cat_li:
    cat_name = cat.find('a').text.strip()
    # No need this url for now
    # cat_url = cat.find('a')['href']
    # print(cat_name)
    sub_cat_list = []
    sub_cat_li = cat.find_all('li', class_='level1')
    for sub_cat in sub_cat_li:
        sub_cat_name = sub_cat.find('a').text.strip()
        sub_cat_url = sub_cat.find('a')['href']
        # print(sub_cat_name, sub_cat_url)

        sub_cat_list.append({
            "category": cat_name,
            "sub_category": sub_cat_name,
            "url": sub_cat_url
        })
    category_list.extend(sub_cat_list)
    print(f"Collected {len(sub_cat_list)} from {cat_name}")

len(category_list)

Collected 8 from Clothing
Collected 12 from Tactical & Duty Gear
Collected 5 from Shooting & Accessories
Collected 6 from Survival & Outdoor
Collected 5 from Firearms
Collected 6 from Knives
Collected 0 from Brands


42

In [18]:
category_list

[{'category': 'Clothing',
  'sub_category': 'Tops',
  'url': 'https://www.krollcorp.com/clothing/tops.html'},
 {'category': 'Clothing',
  'sub_category': 'Bottoms',
  'url': 'https://www.krollcorp.com/clothing/bottoms.html'},
 {'category': 'Clothing',
  'sub_category': 'Baselayer',
  'url': 'https://www.krollcorp.com/clothing/baselayer.html'},
 {'category': 'Clothing',
  'sub_category': 'Outerwear',
  'url': 'https://www.krollcorp.com/clothing/outerwear.html'},
 {'category': 'Clothing',
  'sub_category': 'Footwear',
  'url': 'https://www.krollcorp.com/clothing/footwear.html'},
 {'category': 'Clothing',
  'sub_category': 'Gloves',
  'url': 'https://www.krollcorp.com/clothing/gloves.html'},
 {'category': 'Clothing',
  'sub_category': 'Headwear',
  'url': 'https://www.krollcorp.com/clothing/headwear.html'},
 {'category': 'Clothing',
  'sub_category': 'Clothing Accessories',
  'url': 'https://www.krollcorp.com/clothing/clothing-accessories.html'},
 {'category': 'Tactical & Duty Gear',
  's

In [14]:
from legacy.util.file import save_json_to_dir, load_json_from_dir

In [19]:
save_json_to_dir(category_list, "kroll.json")

In [21]:
loaded=load_json_from_dir("kroll.json")

category_list == loaded

True

In [9]:
from legacy.scraper.kroll import scrape_kroll 

In [10]:
import timeit
from time import sleep

t1 = timeit.default_timer()

cycles = 0

dfs = {}
for i, c in enumerate(category_list,1):
    # print(i)
    print(c)
    url = c.get('url')
    df = scrape_kroll(url)
    dfs[i] = df
    cycles += 1
    sleep(1)
    # break

t2 = timeit.default_timer()

elapsed = t2-t1
print(elapsed)

print(f"Elapsed time: {elapsed:.4f} seconds")
print(f"Elapsed time: {elapsed - cycles} without sleep")

{'category': <li class="level0 nav-1 category-item first level-top parent"><a class="level-top" href="https://www.krollcorp.com/clothing.html"><span>Clothing</span></a><ul class="level0 submenu"><li class="level1 nav-1-1 category-item first parent"><a href="https://www.krollcorp.com/clothing/tops.html"><span>Tops</span></a><ul class="level1 submenu"><li class="level2 nav-1-1-1 category-item first"><a href="https://www.krollcorp.com/clothing/tops/short-sleeve.html"><span>Short Sleeve</span></a></li><li class="level2 nav-1-1-2 category-item"><a href="https://www.krollcorp.com/clothing/tops/long-sleeve.html"><span>Long Sleeve</span></a></li><li class="level2 nav-1-1-3 category-item"><a href="https://www.krollcorp.com/clothing/tops/polos.html"><span>Polos</span></a></li><li class="level2 nav-1-1-4 category-item"><a href="https://www.krollcorp.com/clothing/tops/hoodies-sweatshirts.html"><span>Hoodies &amp; Sweatshirts</span></a></li><li class="level2 nav-1-1-5 category-item last"><a href="h

<string>:1: SyntaxWarning: invalid escape sequence '\/'
<string>:1: SyntaxWarning: invalid escape sequence '\/'
<string>:1: SyntaxWarning: invalid escape sequence '\/'
<string>:1: SyntaxWarning: invalid escape sequence '\/'
<string>:1: SyntaxWarning: invalid escape sequence '\/'
<string>:1: SyntaxWarning: invalid escape sequence '\/'
<string>:1: SyntaxWarning: invalid escape sequence '\/'
<string>:1: SyntaxWarning: invalid escape sequence '\/'
<string>:1: SyntaxWarning: invalid escape sequence '\/'
<string>:1: SyntaxWarning: invalid escape sequence '\/'
<string>:1: SyntaxWarning: invalid escape sequence '\/'
<string>:1: SyntaxWarning: invalid escape sequence '\/'
<string>:1: SyntaxWarning: invalid escape sequence '\/'
<string>:1: SyntaxWarning: invalid escape sequence '\/'


Found 2 objects.
{'category': <li class="level0 nav-1 category-item first level-top parent"><a class="level-top" href="https://www.krollcorp.com/clothing.html"><span>Clothing</span></a><ul class="level0 submenu"><li class="level1 nav-1-1 category-item first parent"><a href="https://www.krollcorp.com/clothing/tops.html"><span>Tops</span></a><ul class="level1 submenu"><li class="level2 nav-1-1-1 category-item first"><a href="https://www.krollcorp.com/clothing/tops/short-sleeve.html"><span>Short Sleeve</span></a></li><li class="level2 nav-1-1-2 category-item"><a href="https://www.krollcorp.com/clothing/tops/long-sleeve.html"><span>Long Sleeve</span></a></li><li class="level2 nav-1-1-3 category-item"><a href="https://www.krollcorp.com/clothing/tops/polos.html"><span>Polos</span></a></li><li class="level2 nav-1-1-4 category-item"><a href="https://www.krollcorp.com/clothing/tops/hoodies-sweatshirts.html"><span>Hoodies &amp; Sweatshirts</span></a></li><li class="level2 nav-1-1-5 category-item

<string>:1: SyntaxWarning: invalid escape sequence '\/'
<string>:1: SyntaxWarning: invalid escape sequence '\/'
<string>:1: SyntaxWarning: invalid escape sequence '\/'
<string>:1: SyntaxWarning: invalid escape sequence '\/'
<string>:1: SyntaxWarning: invalid escape sequence '\/'
<string>:1: SyntaxWarning: invalid escape sequence '\/'
<string>:1: SyntaxWarning: invalid escape sequence '\/'
<string>:1: SyntaxWarning: invalid escape sequence '\/'
<string>:1: SyntaxWarning: invalid escape sequence '\/'
<string>:1: SyntaxWarning: invalid escape sequence '\/'
<string>:1: SyntaxWarning: invalid escape sequence '\/'
<string>:1: SyntaxWarning: invalid escape sequence '\/'
<string>:1: SyntaxWarning: invalid escape sequence '\/'


Found 2 objects.
{'category': <li class="level0 nav-1 category-item first level-top parent"><a class="level-top" href="https://www.krollcorp.com/clothing.html"><span>Clothing</span></a><ul class="level0 submenu"><li class="level1 nav-1-1 category-item first parent"><a href="https://www.krollcorp.com/clothing/tops.html"><span>Tops</span></a><ul class="level1 submenu"><li class="level2 nav-1-1-1 category-item first"><a href="https://www.krollcorp.com/clothing/tops/short-sleeve.html"><span>Short Sleeve</span></a></li><li class="level2 nav-1-1-2 category-item"><a href="https://www.krollcorp.com/clothing/tops/long-sleeve.html"><span>Long Sleeve</span></a></li><li class="level2 nav-1-1-3 category-item"><a href="https://www.krollcorp.com/clothing/tops/polos.html"><span>Polos</span></a></li><li class="level2 nav-1-1-4 category-item"><a href="https://www.krollcorp.com/clothing/tops/hoodies-sweatshirts.html"><span>Hoodies &amp; Sweatshirts</span></a></li><li class="level2 nav-1-1-5 category-item

...

{'category': <li class="level0 nav-6 category-item last level-top parent"><a class="level-top" href="https://www.krollcorp.com/knives.html"><span>Knives</span></a><ul class="level0 submenu"><li class="level1 nav-6-1 category-item first"><a href="https://www.krollcorp.com/knives/automatics.html"><span>Automatics</span></a></li><li class="level1 nav-6-2 category-item"><a href="https://www.krollcorp.com/knives/fixed.html"><span>Fixed</span></a></li><li class="level1 nav-6-3 category-item"><a href="https://www.krollcorp.com/knives/folding.html"><span>Folding</span></a></li><li class="level1 nav-6-4 category-item"><a href="https://www.krollcorp.com/knives/multi-tools.html"><span>Multi-Tools</span></a></li><li class="level1 nav-6-5 category-item"><a href="https://www.krollcorp.com/knives/sharpening-tools.html"><span>Sharpening Tools</span></a></li><li class="level1 nav-6-6 category-item last"><a href="https://www.krollcorp.com/knives/other-blades-accessories.html"><span>Other Blades &amp; Ac

<string>:1: SyntaxWarning: invalid escape sequence '\/'
<string>:1: SyntaxWarning: invalid escape sequence '\/'
<string>:1: SyntaxWarning: invalid escape sequence '\/'
<string>:1: SyntaxWarning: invalid escape sequence '\/'
<string>:1: SyntaxWarning: invalid escape sequence '\/'
<string>:1: SyntaxWarning: invalid escape sequence '\/'
<string>:1: SyntaxWarning: invalid escape sequence '\/'
<string>:1: SyntaxWarning: invalid escape sequence '\/'
<string>:1: SyntaxWarning: invalid escape sequence '\/'
<string>:1: SyntaxWarning: invalid escape sequence '\/'
<string>:1: SyntaxWarning: invalid escape sequence '\/'
<string>:1: SyntaxWarning: invalid escape sequence '\/'
<string>:1: SyntaxWarning: invalid escape sequence '\/'


Found 2 objects.
{'category': <li class="level0 nav-6 category-item last level-top parent"><a class="level-top" href="https://www.krollcorp.com/knives.html"><span>Knives</span></a><ul class="level0 submenu"><li class="level1 nav-6-1 category-item first"><a href="https://www.krollcorp.com/knives/automatics.html"><span>Automatics</span></a></li><li class="level1 nav-6-2 category-item"><a href="https://www.krollcorp.com/knives/fixed.html"><span>Fixed</span></a></li><li class="level1 nav-6-3 category-item"><a href="https://www.krollcorp.com/knives/folding.html"><span>Folding</span></a></li><li class="level1 nav-6-4 category-item"><a href="https://www.krollcorp.com/knives/multi-tools.html"><span>Multi-Tools</span></a></li><li class="level1 nav-6-5 category-item"><a href="https://www.krollcorp.com/knives/sharpening-tools.html"><span>Sharpening Tools</span></a></li><li class="level1 nav-6-6 category-item last"><a href="https://www.krollcorp.com/knives/other-blades-accessories.html"><span>Othe

<string>:1: SyntaxWarning: invalid escape sequence '\/'
<string>:1: SyntaxWarning: invalid escape sequence '\/'
<string>:1: SyntaxWarning: invalid escape sequence '\/'
<string>:1: SyntaxWarning: invalid escape sequence '\/'
<string>:1: SyntaxWarning: invalid escape sequence '\/'
<string>:1: SyntaxWarning: invalid escape sequence '\/'
<string>:1: SyntaxWarning: invalid escape sequence '\/'
<string>:1: SyntaxWarning: invalid escape sequence '\/'
<string>:1: SyntaxWarning: invalid escape sequence '\/'
<string>:1: SyntaxWarning: invalid escape sequence '\/'
<string>:1: SyntaxWarning: invalid escape sequence '\/'
<string>:1: SyntaxWarning: invalid escape sequence '\/'
<string>:1: SyntaxWarning: invalid escape sequence '\/'


69.79232272296213
Elapsed time: 69.7923 seconds
Elapsed time: 27.792322722962126 without sleep


In [11]:
merged_df = pd.concat(dfs.values(), ignore_index=True)

len(merged_df)

497

In [ ]:
# data cleaning

merged_df['category'] = merged_df['item_category']
merged_df['sub_category'] = merged_df['item_category2']

remove_columns = [
    'item_category',
    'item_category2',

    "item_list_id",
    "index",
]

for column in remove_columns:
    if column in df.columns:
        merged_fd = merged_df.drop(columns=[column])


# merged_df = merged_df.drop([, 'item_category2'], axis=1)

In [13]:
merged_df.head()

,item_name,item_id,price,item_brand,item_list_id,index,category,sub_category
0,UA Boxed Sportstyle Short Sleeve T-Shirt,1329581,14.29,Under Armour,4,1,Clothing,Tops
1,Men's UA Tac Elite Polo,1379202,37.14,Under Armour,4,2,Clothing,Tops
2,Women's UA Tac Elite Polo,1379206,37.14,Under Armour,4,3,Clothing,Tops
3,Men's UA Tech ABC Camo Short Sleeve,1361698,20.00,Under Armour,4,4,Clothing,Tops
4,Men's UA Rival Fleece Logo Hoodie,1379758,31.43,Under Armour,4,5,Clothing,Tops


In [14]:
from legacy.util.gsheet import update_sheet_by_id
from config import SheetName

In [15]:
merged_df.columns

Index(['item_name', 'item_id', 'price', 'item_brand', 'item_list_id', 'index',
       'category', 'sub_category'],
      dtype='object')

In [16]:
sheet_id = "1ZE7BeE26iM43x_ympaIgOdl6lXWrwXg-PZZAKuIBEIw"

update_sheet_by_id(merged_df, sheet_id, SheetName.KROLL.value)